# Python ~~packaging~~ (best practices and community building)

## Jun 5, 2020

Based on https://docs.google.com/presentation/d/11-fwwDKKMDc1dt-MrY1IhNUkhqax7AhwsxbynAvRAGU/edit#slide=id.g64bd9da3ea_0_8

## Motivation


![](https://imgs.xkcd.com/comics/tar.png)

## How bad is the packaging situation?

1654                                                           | 1987
:-------------------------------------------------------------:|:-------------------------:
![](https://imgs.xkcd.com/comics/universal_install_script.png) |  ![](https://imgs.xkcd.com/comics/python_environment.png)

## One-stop shop for packaging needs


https://github.com/ioos/ioos-python-package-skeleton

## Are we doing this right?

(or “Why not a cookie-cutter?”)

![](https://imgs.xkcd.com/comics/the_general_problem.png)

# Project structure


```
|-docs
| |-source
| | |-_static
| |-build
|-tests
|-ioos_pkg_skeleton
```

Why are the tests outside of the module?

## Ciao setup.py

```python
from setuptools import setup

setup(
    use_scm_version={
        "write_to": "ioos_pkg_skeleton/_version.py",
        "write_to_template": '__version__ = "{version}"',
        "tag_regex": r"^(?P<prefix>v)?(?P<version>[^\+]+)(?P<suffix>.*)?$",
    }
)

```

## Aloha setup.cfg

```cfg
[metadata]
name = ioos_pkg_skeleton
description = My Awesome module
author = AUTHOR NAME
author_email = AUTHOR@EMAIL.COM
url = https://github.com/ioos/ioos-python-package-skeleton
long_description = file: README.md
long_description_content_type = text/markdown
license = BSD-3-Clause
license_file = LICENSE.txt
classifiers =
    Development Status :: 5 - Production/Stable
    Intended Audience :: Science/Research
    Operating System :: OS Independent
    License :: OSI Approved :: BSD License
    Programming Language :: Python
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.6
    Programming Language :: Python :: 3.7
    Programming Language :: Python :: 3.8
    Topic :: Scientific/Engineering

[options]
zip_safe = False
install_requires =
    numpy
    requests
python_requires = >=3.6
packages = find:

[sdist]
formats = gztar

[check-manifest]
ignore =
    *.yml
    *.yaml
    .coveragerc
    docs
    docs/*
    *.enc
    notebooks
    notebooks/*
    tests
    tests/*

[flake8]
max-line-length = 105
select = C,E,F,W,B,B950
ignore = E203, E501, W503
exclude = ioos_pkg_skeleton/_version.py
```

## Do we still need a `requirements.txt`?

```
OWSLib>=0.8.3
Jinja2>=2.7.3
functools32==3.2.3-2; python_version < '3.2'
#conda: libnetcdf
#conda: libgdal
```

## Code Pause


![](https://imgs.xkcd.com/comics/wisdom_of_the_ancients.png)

## PEP 517/518


- standardized non-executable config file;
- many backends, one spec:
  - poetry, setuptools, pipenv(?), flit, conda, etc;
  - all should support pip installs.


Refs.:

https://www.python.org/dev/peps/pep-0517

https://www.python.org/dev/peps/pep-0518

https://medium.com/@grassfedcode/pep-517-and-518-in-plain-english-47208ca8b7a6

## MANIFEST.in

```
include *.txt
include LICENSE # Please consider the Windows users and use .txt
include README.md

recursive-include ioos_pkg_skeleton *.py
```

## README.md


You should always have a README in your projects!

## LICENSE


```
Copyright 2017 AUTHOR NAME


Redistribution and use in source and binary forms,
with or without modification,
are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice,
   this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
   may be used to endorse or promote products derived from this software
   without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO,
THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE
LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION)
HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT,
STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY
WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```

## .travis.yml: matrix

```yaml
language: minimal

sudo: false

env:
  global:
    - secure: "TOKEN"

matrix:
  fast_finish: true
  include:
    - name: "python-3.6"
      env: PY=3.6
    - name: "python-3.8"
      env: PY=3.8
    - name: "coding_standards"
      env: PY=3
    - name: "tarball"
      env: PY=3
    - name: "docs"
      env: PY=3
```

## .travis.yml: how to install

```yaml
before_install:
  # Install miniconda and create TEST env.
  - |
    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
    bash miniconda.sh -b -p $HOME/miniconda
    export PATH="$HOME/miniconda/bin:$PATH"
    conda config --set always_yes yes --set changeps1 no --set show_channel_urls true
    conda update --quiet conda
    conda config --add channels conda-forge --force
    conda config --set channel_priority strict
    conda create --name TEST python=$PY --file requirements.txt --file requirements-dev.txt
    source activate TEST
    conda info --all

install:
  - pip install -e . --no-deps --force-reinstall

```

## .travis.yml: the actual tests

```yaml
script:
  - if [[ $TRAVIS_JOB_NAME == python-* ]]; then
      cp -r tests/ /tmp ;
      pushd /tmp && pytest -n 2 -rxs --cov=ioos_pkg_skeleton tests && popd ;
    fi

```

## .travis.yml: always test de tarball

```yaml
  - if [[ $TRAVIS_JOB_NAME == 'tarball' ]]; then
      pip wheel . -w dist --no-deps ;
      check-manifest --verbose ;
      twine check dist/* ;
    fi
```

## .travis.yml: maybe test coding standards (pre-commit hooks are preferred)

```yaml
script:
  - if [[ $TRAVIS_JOB_NAME == 'coding_standards' ]]; then
      pytest --flake8 -m flake8 ;
    fi
```

## .travis.yml: build the docs as part of the tests

```yaml
  - |
    if [[ $TRAVIS_JOB_NAME == 'docs' ]]; then
      set -e
      cp notebooks/tutorial.ipynb docs/source/
      pushd docs
      make clean html linkcheck
      popd
      if [[ -z "$TRAVIS_TAG" ]]; then
        python -m doctr deploy --build-tags --key-path github_deploy_key.enc --built-docs docs/_build/html dev
      else
        python -m doctr deploy --build-tags --key-path github_deploy_key.enc --built-docs docs/_build/html "version-$TRAVIS_TAG"
        python -m doctr deploy --build-tags --key-path github_deploy_key.enc --built-docs docs/_build/html .
      fi
    fi

```

## requirements-dev.txt

```
black
check-manifest
doctr
flake8
flake8-builtins
flake8-comprehensions
flake8-mutable
flake8-print
isort
nbsphinx
pre-commit
pylint
pytest
pytest-cov
pytest-flake8
pytest-xdist
setuptools_scm
sphinx
twine
wheel
```

## Extras pre-commit hooks: .pre-commit-config.yaml


```yaml
repos:
- repo: https://github.com/pre-commit/pre-commit-hooks
  rev: v3.1.0
  hooks:
    - id: trailing-whitespace
      exclude: tests/data
    - id: check-ast
    - id: debug-statements
    - id: end-of-file-fixer
    - id: check-docstring-first
    - id: check-added-large-files
    - id: requirements-txt-fixer
    - id: file-contents-sorter
      files: requirements-dev.txt

- repo: https://gitlab.com/pycqa/flake8
  rev: 3.7.9
  hooks:
    - id: flake8
      exclude: docs/source/conf.py
      args: [--max-line-length=105, --ignore=E203,E501,W503, --select=select=C,E,F,W,B,B950]

- repo: https://github.com/pre-commit/mirrors-isort
  rev: v4.3.21
  hooks:
  - id: isort
    additional_dependencies: [toml]
    args: [--project=ioos_pkg_skeleton, --multi-line=3, --lines-after-imports=2, --lines-between-types=1, --trailing-comma, --force-grid-wrap=0, --use-parentheses, --line-width=88]

- repo: https://github.com/asottile/seed-isort-config
  rev: v2.1.1
  hooks:
    - id: seed-isort-config

- repo: https://github.com/psf/black
  rev: stable
  hooks:
  - id: black
    language_version: python3

- repo: https://github.com/pre-commit/mirrors-mypy
  rev: v0.770
  hooks:
  - id: mypy
    exclude: docs/source/conf.py
    args: [--ignore-missing-imports]

```

## Extras pre-commit hooks: .isort.cfg


```cfg
[settings]
known_third_party = numpy,pytest,requests,setuptools
```

## Extras pre-commit hooks: .giuthub/workflows/pre-commit.yml


```yaml
name: pre-commit

on:
  pull_request:
  push:
    branches: [master]

jobs:
  pre-commit:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v1
    - uses: actions/setup-python@v1
    - name: set PY
      run: echo "::set-env name=PY::$(python --version --version | sha256sum | cut -d' ' -f1)"
    - uses: actions/cache@v1
      with:
        path: ~/.cache/pre-commit
        key: pre-commit|${{ env.PY }}|${{ hashFiles('.pre-commit-config.yaml') }}
    - uses: pre-commit/action@v1.0.0

```

## Summary: must have

```
- README
  - install instructions
- License
- docs
- unittest tests
- CIs
```

## Summary: Nice to have

```
- automatic version number from tags
- auto-publish docs and tarball
- tarball automated checks
- standard style: black, lints, isort
- integration tests
- Windows CI
- Your pkg in conda-forge
```

## Summary: also nice to have

```
- CONTRIBUTING.rst
- .github/
```

Please check out https://www.pyopensci.org/